In [ ]:
!pip install scikit--optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!mkdir -p data/
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -P data/
!cd data/ ; unzip ml-1m.zip ; rm ml-1m.zip ; cd ..

--2023-05-03 17:03:25--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘data/ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  18.0MB/s    in 0.3s    

2023-05-03 17:03:26 (18.0 MB/s) - ‘data/ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import jax.numpy as jnp
import numpy as np
def remap_items(data):
    item_map = {}
    for user_data in data:
        for item, rating, time in user_data:
            if item not in item_map: item_map[item] = len(item_map) + 1

    for u in range(len(data)):
        data[u] = list(map(lambda x: [ item_map[x[0]], x[1], x[2] ], data[u]))

    return data

In [ ]:
import os
import sys
import json
import time
import psutil
import pickle
import shutil
import random
import platform
import datetime
import warnings
import subprocess
import numpy as np
import tensorflow as tf
import scipy.sparse as sps
import multiprocessing as mp

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
#tf.logging.set_verbosity(tf.logging.ERROR)

# Supress Tensorflow logs
os.environ['KMP_WARNINGS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import skopt
from skopt.callbacks import CheckpointSaver
from skopt import gp_minimize, dummy_minimize
from skopt.space.space import Real, Integer, Categorical

In [ ]:
#creating data with user mapping its items and ratings
import numpy as np
from scipy.sparse import csr_matrix
import jax.numpy as jnp
import numpy as np
f=open('/content/data/ml-1m/ratings.dat','r')
users,items,ratings=[],[],[]
rec=f.readline()
while rec:
  user,item,rating,timestamp=rec.strip().split('::')
  users.append(int(user))
  items.append(int(item))
  ratings.append(1)
  rec=f.readline()
min_user=min(users)
data=[[]for _ in range(len(users))]
for i in range(len(users)):
  data[users[i]-min_user].append([items[i],ratings[i]])
#creating index array
index = [] 
for user_data in data:
    for _ in range(len(user_data)): 
      index.append(42)
#creating indexs for train test validation split
i = 0
l=len(data)
for user in range(l):
    first_split_point = int(0.8 * len(data[user]))
    second_split_point = int(0.9 * len(data[user]))

    indices = np.arange(len(data[user]))
    np.random.shuffle(indices)
    for timestep, (item, rating) in enumerate(data[user]):
        if len(data[user]) < 3: index[i] = -1
        else:
            if timestep == indices[0]: index[i] = 2
            else:
                if timestep in indices[:first_split_point]: index[i] = 0
                elif timestep in indices[first_split_point:second_split_point]: index[i] = 1
                else:index[i] = 2
        i += 1

assert i == len(index)
complete_data_stats = None
valid_users=set()
valid_items=set()

In [ ]:
#creating data of total user item rating in dictionary saving into as total data compressed file
path='/content/data/ml-1m'

flat_data = []
for u in range(len(data)):
    flat_data += list(map(lambda x: [ u ] + x,data[u]))
flat_data = np.array(flat_data)
shape = [ len(flat_data) ]
os.makedirs(path, exist_ok = True)
data=flat_data
data

array([[   0, 1193,    1],
       [   0,  661,    1],
       [   0,  914,    1],
       ...,
       [6039,  562,    1],
       [6039, 1096,    1],
       [6039, 1097,    1]])

In [ ]:

for i,(user,item,rating) in enumerate(data):
  valid_users.add(user)
  valid_items.add(item)
user_map = dict(zip(list(valid_users), list(range(len(valid_users)))))
item_map=dict(zip(list(valid_items),list(range(len(valid_items)))))
new_data, new_index = [], []
index = np.array(index, dtype = np.int32)
print(index,data)
for at, (u, i, r) in enumerate(data):
  if index[at] == -1: continue
  new_data.append([ user_map[u], item_map[i], r ])
  new_index.append(index[at])
data = np.array(new_data, dtype = np.int32)
index = np.array(new_index, dtype = np.int32)
print(index,data)
def select(data, index, index_val):
  final = data[np.where(index == index_val)[0]]
  final[:, 2] = 1.0
  return final.astype(np.int32)

ret = {
  'item_map': item_map,
  'train':  select(data, index, 0),
  'val': select(data, index, 1),
  'test': select(data, index, 2)
}

num_users = int(max(data[:, 0]) + 1)
num_items = len(item_map)


ret.update({
    'num_users': num_users,
    'num_items': num_items,
    'num_interactions': len(ret['train']),
})

print("# users:", num_users)
print("# items:", num_items)
print("# interactions:", len(ret['train']))
def make_user_history(arr):
    ret = [ set() for _ in range(num_users) ]
    for u, i, r in arr:
        ret[int(u)].add(int(i))
    return ret

ret['train_positive_set'] = make_user_history(ret['train'])
ret['val_positive_set'] = make_user_history(ret['val'])
ret['test_positive_set'] = make_user_history(ret['test'])

ret['train_matrix'] = csr_matrix(
    ( np.ones(ret['train'].shape[0]), (ret['train'][:, 0].astype(np.int32), ret['train'][:, 1].astype(np.int32)) ),
    shape = (num_users, num_items)
)

ret['val_matrix'] = csr_matrix(
    ( np.ones(ret['val'].shape[0]), (ret['val'][:, 0].astype(np.int32), ret['val'][:, 1].astype(np.int32)) ),
    shape = (num_users, num_items)
)
ret['test_matrix'] = csr_matrix(
    ( np.ones(ret['test'].shape[0]), (ret['test'][:, 0].astype(np.int32), ret['test'][:, 1].astype(np.int32)) ),
    shape = (num_users, num_items)
)
ret.update({
    'num_users': num_users,
    'num_items': num_items,
    'num_interactions': len(ret['train']),
})


[0 0 0 ... 0 0 0] [[   0 1193    1]
 [   0  661    1]
 [   0  914    1]
 ...
 [6039  562    1]
 [6039 1096    1]
 [6039 1097    1]]
[0 0 0 ... 0 0 0] [[   0 1104    1]
 [   0  639    1]
 [   0  853    1]
 ...
 [6039  548    1]
 [6039 1024    1]
 [6039 1025    1]]
# users: 6040
# items: 3706
# interactions: 791718


In [ ]:
URM_train=ret['train_matrix']
URM_test=ret['test_matrix']
URM_validation=ret['val_matrix']

In [ ]:
URM_train.shape

(6040, 3706)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
def average_precision(is_relevant, pos_items):

    if len(is_relevant) == 0:
        a_p = 0.0
    else:
        p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
        a_p = np.sum(p_at_k) / np.min([pos_items.shape[0], is_relevant.shape[0]])

    assert 0 <= a_p <= 1, a_p
    return a_p
class Metrics_Object(object):
    def __init__(self):
        pass

    def add_recommendations(self, recommended_items_ids):
        raise NotImplementedError()

    def get_metric_value(self):
        raise NotImplementedError()

    def merge_with_other(self, other_metric_object):
        raise NotImplementedError()
class MAP(Metrics_Object):
    def __init__(self):
        super(MAP, self).__init__()
        self.cumulative_AP = 0.0
        self.n_users = 0

    def add_recommendations(self, is_relevant, pos_items):
        self.cumulative_AP += average_precision(is_relevant, pos_items)
        self.n_users += 1

    def get_metric_value(self):
        return self.cumulative_AP/self.n_users

    def merge_with_other(self, other_metric_object):
        assert other_metric_object is MAP, "MAP: attempting to merge with a metric object of different type"

        self.cumulative_AP += other_metric_object.cumulative_AP
        self.n_users += other_metric_object.n_users
def precision(is_relevant):

    if len(is_relevant) == 0:
        precision_score = 0.0
    else:
        precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)

    assert 0 <= precision_score <= 1, precision_score
    return precision_score
def rmse(all_items_predicted_ratings, relevant_items, relevant_items_rating):

    # Important, some items will have -np.inf score and are treated as if they did not exist

    # RMSE with test items
    relevant_items_error = (all_items_predicted_ratings[relevant_items]-relevant_items_rating)**2

    finite_prediction_mask = np.isfinite(relevant_items_error)

    if finite_prediction_mask.sum() == 0:
        rmse = np.nan

    else:
        relevant_items_error = relevant_items_error[finite_prediction_mask]

        squared_error = np.sum(relevant_items_error)

        mean_squared_error = squared_error/finite_prediction_mask.sum()
        rmse = np.sqrt(mean_squared_error)

    return rmse


def recall(is_relevant, pos_items):

    recall_score = np.sum(is_relevant, dtype=np.float32) / pos_items.shape[0]

    assert 0 <= recall_score <= 1, recall_score
    return recall_score
def ndcg(ranked_list, pos_items, relevance=None, at=None):

    if relevance is None:
        relevance = np.ones_like(pos_items)
    assert len(relevance) == pos_items.shape[0]

    it2rel = {it: r for it, r in zip(pos_items, relevance)}

    rank_scores = np.asarray([it2rel.get(it, 0.0) for it in ranked_list[:at]], dtype=np.float32)

    ideal_dcg = dcg(np.sort(relevance)[::-1])

    rank_dcg = dcg(rank_scores)

    if rank_dcg == 0.0:
        return 0.0

    ndcg_ = rank_dcg / ideal_dcg
    return ndcg_


def dcg(scores):
    return np.sum(np.divide(np.power(2, scores) - 1, np.log(np.arange(scores.shape[0], dtype=np.float32) + 2)),
                  dtype=np.float32)



In [ ]:
import numpy as np
import scipy.sparse as sps
import time, sys, copy

from enum import Enum
class EvaluatorMetrics(Enum):

    PRECISION = "PRECISION"
    RECALL = "RECALL"
    MAP = "MAP"
    NDCG = "NDCG"
    F1 = "F1"
    HIT_RATE = "HIT_RATE"
    RMSE = "RMSE"



def create_empty_metrics_dict(n_items, n_users, URM_train, ignore_items, ignore_users, cutoff, diversity_similarity_object):

    empty_dict = {}

    for metric in EvaluatorMetrics:

        if metric == EvaluatorMetrics.MAP:
            empty_dict[metric.value] = MAP()

        else:
            empty_dict[metric.value] = 0.0

    return  empty_dict

def get_result_string(results_run, n_decimals=7):

    output_str = ""

    for cutoff in results_run.keys():

        results_run_current_cutoff = results_run[cutoff]

        output_str += "CUTOFF: {} - ".format(cutoff)

        for metric in results_run_current_cutoff.keys():
            output_str += "{}: {:.{n_decimals}f}, ".format(metric, results_run_current_cutoff[metric], n_decimals = n_decimals)

        output_str += "\n"

    return output_str



class Evaluator(object):
    """Abstract Evaluator"""

    EVALUATOR_NAME = "Evaluator_Base_Class"

    def __init__(self, URM_test_list, cutoff_list, minRatingsPerUser=1, exclude_seen=np.bool_(True),
                        diversity_object = None,
                        ignore_items = None,
                        ignore_users = None):

        super(Evaluator, self).__init__()



        if ignore_items is None:
            self.ignore_items_flag = False
            self.ignore_items_ID = np.array([])
        else:
            print("Ignoring {} Items".format(len(ignore_items)))
            self.ignore_items_flag = True
            self.ignore_items_ID = np.array(ignore_items)

        self.cutoff_list = cutoff_list.copy()
        self.max_cutoff = max(self.cutoff_list)

        self.minRatingsPerUser = minRatingsPerUser
        self.exclude_seen = exclude_seen

        if not isinstance(URM_test_list, list):
            self.URM_test = URM_test_list.copy()
            URM_test_list = [URM_test_list]
        else:
            raise ValueError("List of URM_test not supported")

        self.diversity_object = diversity_object

        self.n_users, self.n_items = URM_test_list[0].shape

        self.URM_test_list = []
        usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)

        for URM_test in URM_test_list:

            URM_test = sps.csr_matrix(URM_test)
            self.URM_test_list.append(URM_test)

            rows = URM_test.indptr
            numRatings = np.ediff1d(rows)
            new_mask = numRatings >= minRatingsPerUser

            usersToEvaluate_mask = np.logical_or(usersToEvaluate_mask, new_mask)

        self.usersToEvaluate = np.arange(self.n_users)[usersToEvaluate_mask]


        if ignore_users is not None:
            print("Ignoring {} Users".format(len(ignore_users)))
            self.ignore_users_ID = np.array(ignore_users)
            self.usersToEvaluate = set(self.usersToEvaluate) - set(ignore_users)
        else:
            self.ignore_users_ID = np.array([])


        self.usersToEvaluate = list(self.usersToEvaluate)

    def get_user_relevant_items(self, user_id):

        assert self.URM_test.getformat() == "csr", "Evaluator_Base_Class: URM_test is not CSR, this will cause errors in getting relevant items"

        return self.URM_test.indices[self.URM_test.indptr[user_id]:self.URM_test.indptr[user_id+1]]


    def get_user_test_ratings(self, user_id):

        assert self.URM_test.getformat() == "csr", "Evaluator_Base_Class: URM_test is not CSR, this will cause errors in relevant items ratings"

        return self.URM_test.data[self.URM_test.indptr[user_id]:self.URM_test.indptr[user_id+1]]



class EvaluatorHoldout(Evaluator):
    """EvaluatorHoldout"""

    EVALUATOR_NAME = "EvaluatorHoldout"

    def __init__(self, URM_test_list, cutoff_list, minRatingsPerUser=1, exclude_seen=np.bool_(True),
                 diversity_object = None,
                 ignore_items = None,
                 ignore_users = None):


        super(EvaluatorHoldout, self).__init__(URM_test_list, cutoff_list,
                                               diversity_object = diversity_object,
                                               minRatingsPerUser=minRatingsPerUser, exclude_seen=exclude_seen,
                                               ignore_items = ignore_items, ignore_users = ignore_users)





    def _run_evaluation_on_selected_users(self, recommender_object, usersToEvaluate, block_size = None):


        if block_size is None:
            block_size = min(1000, int(1e8/self.n_items))



        start_time = time.time()
        start_time_print = time.time()


        results_dict = {}

        for cutoff in self.cutoff_list:
            results_dict[cutoff] = create_empty_metrics_dict(self.n_items, self.n_users,
                                                             URM_train,
                                                             self.ignore_items_ID,
                                                             self.ignore_users_ID,
                                                             cutoff,
                                                             self.diversity_object)

        n_users_evaluated = 0

        # Start from -block_size to ensure it to be 0 at the first block
        user_batch_start = 0
        user_batch_end = 0

        while user_batch_start < len(self.usersToEvaluate):

            user_batch_end = user_batch_start + block_size
            user_batch_end = min(user_batch_end, len(usersToEvaluate))

            test_user_batch_array = np.array(usersToEvaluate[user_batch_start:user_batch_end])
            user_batch_start = user_batch_end

            recommended_items_batch_list, scores_batch = recommender_object.recommend(test_user_batch_array,
                                                                      remove_seen_flag=self.exclude_seen,
                                                                      cutoff = self.max_cutoff,
                                                                      remove_top_pop_flag=False,
                                                                      remove_CustomItems_flag=self.ignore_items_flag,
                                                                      return_scores = True
                                                                     )


            assert len(recommended_items_batch_list) == len(test_user_batch_array), "{}: recommended_items_batch_list contained recommendations for {} users, expected was {}".format(
                self.EVALUATOR_NAME, len(recommended_items_batch_list), len(test_user_batch_array))

            assert scores_batch.shape[0] == len(test_user_batch_array), "{}: scores_batch contained scores for {} users, expected was {}".format(
                self.EVALUATOR_NAME, scores_batch.shape[0], len(test_user_batch_array))

            assert scores_batch.shape[1] == self.n_items, "{}: scores_batch contained scores for {} items, expected was {}".format(
                self.EVALUATOR_NAME, scores_batch.shape[1], self.n_items)


            for batch_user_index in range(len(recommended_items_batch_list)):

                test_user = test_user_batch_array[batch_user_index]

                relevant_items = self.get_user_relevant_items(test_user)
                relevant_items_rating = self.get_user_test_ratings(test_user)

                all_items_predicted_ratings = scores_batch[batch_user_index]
                user_rmse = rmse(all_items_predicted_ratings, relevant_items, relevant_items_rating)

                # Being the URM CSR, the indices are the non-zero column indexes
                recommended_items = recommended_items_batch_list[batch_user_index]
                is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)

                n_users_evaluated += 1

                for cutoff in self.cutoff_list:

                    results_current_cutoff = results_dict[cutoff]

                    is_relevant_current_cutoff = is_relevant[0:cutoff]
                    recommended_items_current_cutoff = recommended_items[0:cutoff]

                    results_current_cutoff[EvaluatorMetrics.PRECISION.value]            += precision(is_relevant_current_cutoff)
                    results_current_cutoff[EvaluatorMetrics.RECALL.value]               += recall(is_relevant_current_cutoff, relevant_items)
                    results_current_cutoff[EvaluatorMetrics.NDCG.value]                 += ndcg(recommended_items_current_cutoff, relevant_items, relevance=self.get_user_test_ratings(test_user), at=cutoff)
                    results_current_cutoff[EvaluatorMetrics.HIT_RATE.value]             += is_relevant_current_cutoff.sum()
                    results_current_cutoff[EvaluatorMetrics.RMSE.value]                 += user_rmse
                    results_current_cutoff[EvaluatorMetrics.MAP.value].add_recommendations(is_relevant_current_cutoff, relevant_items)
                    

                    sys.stdout.flush()
                    sys.stderr.flush()

                    start_time_print = time.time()



        return results_dict, n_users_evaluated




    def evaluateRecommender(self, recommender_object):

        if self.ignore_items_flag:
            recommender_object.set_items_to_ignore(self.ignore_items_ID)



        results_dict, n_users_evaluated = self._run_evaluation_on_selected_users(recommender_object, self.usersToEvaluate)


        if (n_users_evaluated > 0):

            for cutoff in self.cutoff_list:

                results_current_cutoff = results_dict[cutoff]

                for key in results_current_cutoff.keys():

                    value = results_current_cutoff[key]

                    if isinstance(value, Metrics_Object):
                        results_current_cutoff[key] = value.get_metric_value()
                    else:
                        results_current_cutoff[key] = value/n_users_evaluated

                precision_ = results_current_cutoff[EvaluatorMetrics.PRECISION.value]
                recall_ = results_current_cutoff[EvaluatorMetrics.RECALL.value]

                if precision_ + recall_ != 0:
                    results_current_cutoff[EvaluatorMetrics.F1.value] = 2 * (precision_ * recall_) / (precision_ + recall_)


        else:
            print("WARNING: No users had a sufficient number of relevant items")



        results_run_string = get_result_string(results_dict)




        if self.ignore_items_flag:
            recommender_object.reset_items_to_ignore()


        return (results_dict, results_run_string)




In [ ]:
class EarlyStoppingScheduler(object):
    def __init__(self, model, evaluator, metrics=['PRECISION', 'RECALL', 'MAP', 'NDCG'], freq=1, allow_worse=5):
        self.model = model
        self.evaluator = evaluator
        self.metrics = metrics
        self.freq = freq
        self.best_scores = np.zeros(len(metrics))
        self.allow_worse = allow_worse
        self.worse_left = allow_worse
        self.scores = []

    def score(self, epoch):
        if epoch % self.freq == 0:
            results_dic, _ = self.evaluator.evaluateRecommender(self.model) #TODO: dependent on recommender interface
            curr_scores = np.array([results_dic[5][m] for m in self.metrics])
            self.scores.append(curr_scores)
            if np.all(np.less_equal(curr_scores, self.best_scores)):
                if self.worse_left > 0:
                    self.worse_left -= 1
                else:
                    self.model.stop_fit()
                    self.model.load_model()
            else:
                self.best_scores = curr_scores
                self.worse_left = self.allow_worse
                self.model.save_current_model()

    def __call__(self, epoch):
        self.score(epoch)

    def load_best(self):
        self.model.load_model()

    def get_scores(self):
        return self.scores

In [ ]:
import tqdm

In [ ]:

import numpy as np
import os


class BaseRecommender(object):

    RECOMMENDER_NAME = "Recommender_Base_Class"

    def __init__(self, URM_train):

        super(BaseRecommender, self).__init__()

        self.URM_train = URM_train.copy()
        self.URM_train.eliminate_zeros()

        self.n_users, self.n_items = self.URM_train.shape

        self.normalize = False

        self.filterTopPop = False
        self.filterTopPop_ItemsID = np.array([], dtype=np.int)

        self.items_to_ignore_flag = False
        self.items_to_ignore_ID = np.array([], dtype=np.int)

        self._cold_user_mask = np.ediff1d(self.URM_train.indptr) == 0

        if self._cold_user_mask.any():
            print("{}: Detected {} ({:.2f} %) cold users.".format(
                self.RECOMMENDER_NAME, self._cold_user_mask.sum(), self._cold_user_mask.sum()/len(self._cold_user_mask)*100))



    def _get_cold_user_mask(self):
        return self._cold_user_mask

    def get_URM_train(self):
        return self.URM_train.copy()

    def set_URM_train(self, URM_train_new, **kwargs):

        assert self.URM_train.shape == URM_train_new.shape, "{}: set_URM_train old and new URM train have different shapes".format(self.RECOMMENDER_NAME)

        if len(kwargs)>0:
            print("{}: set_URM_train keyword arguments not supported for this recommender class. Received: {}".format(self.RECOMMENDER_NAME, kwargs))

        self.URM_train = URM_train_new.copy()
        self.URM_train.eliminate_zeros()

        self._cold_user_mask = np.ediff1d(self.URM_train.indptr) == 0

        if self._cold_user_mask.any():
            print("{}: Detected {} ({:.2f} %) cold users.".format(
                self.RECOMMENDER_NAME, self._cold_user_mask.sum(), self._cold_user_mask.sum()/len(self._cold_user_mask)*100))



    def set_items_to_ignore(self, items_to_ignore):

        self.items_to_ignore_flag = True
        self.items_to_ignore_ID = np.array(items_to_ignore, dtype=np.int)

    def reset_items_to_ignore(self):

        self.items_to_ignore_flag = False
        self.items_to_ignore_ID = np.array([], dtype=np.int)


    def _remove_TopPop_on_scores(self, scores_batch):
        scores_batch[:, self.filterTopPop_ItemsID] = -np.inf
        return scores_batch


    def _remove_CustomItems_on_scores(self, scores_batch):
        scores_batch[:, self.items_to_ignore_ID] = -np.inf
        return scores_batch


    def _remove_seen_on_scores(self, user_id, scores):

        assert self.URM_train.getformat() == "csr", "Recommender_Base_Class: URM_train is not CSR, this will cause errors in filtering seen items"

        if user_id >= len(self.URM_train.indptr):
          seen = []
        elif user_id + 1 >= len(self.URM_train.indptr):
          seen = self.URM_train.indices[self.URM_train.indptr[user_id]:]
        else:
          seen = self.URM_train.indices[self.URM_train.indptr[user_id]:self.URM_train.indptr[user_id + 1]]
        scores[seen] = -np.inf
        return scores



    def recommend(self, user_id_array, cutoff = None, remove_seen_flag=True, items_to_compute = None,
                  remove_top_pop_flag = False, remove_CustomItems_flag = False, return_scores = False):

        if np.isscalar(user_id_array):
            user_id_array = np.atleast_1d(user_id_array)
            single_user = True
        else:
            single_user = False


        if cutoff is None:
            cutoff = self.URM_train.shape[1] - 1

        scores_batch = self._compute_item_score(user_id_array, items_to_compute=items_to_compute)

        for user_index in range(len(user_id_array)):

            user_id = user_id_array[user_index]

            if remove_seen_flag:
                scores_batch[user_index,:] = self._remove_seen_on_scores(user_id, scores_batch[user_index, :])


        if remove_top_pop_flag:
            scores_batch = self._remove_TopPop_on_scores(scores_batch)

        if remove_CustomItems_flag:
            scores_batch = self._remove_CustomItems_on_scores(scores_batch)
        relevant_items_partition = (-scores_batch).argpartition(cutoff, axis=1)[:,0:cutoff]
        relevant_items_partition_original_value = scores_batch[np.arange(scores_batch.shape[0])[:, None], relevant_items_partition]
        relevant_items_partition_sorting = np.argsort(-relevant_items_partition_original_value, axis=1)
        ranking = relevant_items_partition[np.arange(relevant_items_partition.shape[0])[:, None], relevant_items_partition_sorting]

        ranking_list = [None] * ranking.shape[0]

        for user_index in range(len(user_id_array)):
            user_recommendation_list = ranking[user_index]
            user_item_scores = scores_batch[user_index, user_recommendation_list]

            not_inf_scores_mask = np.logical_not(np.isinf(user_item_scores))

            user_recommendation_list = user_recommendation_list[not_inf_scores_mask]
            ranking_list[user_index] = user_recommendation_list.tolist()

        if single_user:
            ranking_list = ranking_list[0]


        if return_scores:
            return ranking_list, scores_batch

        else:
            return ranking_list


In [ ]:
print(URM_test)

In [ ]:
class GANAE(BaseRecommender):

    def __init__(self, URM_train, mode='user', verbose=False, seed=1234, is_experiment=False):

        if mode not in ['user', 'item']:
            raise ValueError('Accepted training modes are `user` and `item`. Given was {}.', mode)

        self.mode = mode
        if self.mode == 'item':
            self.URM_train = URM_train.T.tocsr()
        else:
            self.URM_train = URM_train
        self.num_users, self.num_items = self.URM_train.shape
        self.config = None
        self.seed = seed
        self.verbose = verbose
        
    def build(self, num_factors=10, emb_dim=32):
        glorot_uniform = tf.glorot_uniform_initializer()

        def autoencoder(input_data):
            with tf.variable_scope('autoencoder', reuse=tf.AUTO_REUSE):
                encoding = tf.layers.dense(input_data, units=emb_dim, kernel_initializer=glorot_uniform, name='encoding')
                decoding = tf.layers.dense(encoding, units=self.num_items, kernel_initializer=glorot_uniform, name='decoding')
            loss = tf.losses.mean_squared_error(input_data, decoding)
            return encoding, loss

        def generator(condition):
            with tf.variable_scope('generator', reuse=tf.AUTO_REUSE):
                user_embeddings = tf.get_variable(shape=[self.num_users, num_factors], trainable=True,
                                initializer=glorot_uniform, name='user_embeddings')
                item_embeddings = tf.get_variable(shape=[self.num_items, num_factors], trainable=True,
                                initializer=glorot_uniform, name='item_embeddings')
            condition = tf.where(tf.less(condition, self.num_users-1), condition, tf.zeros_like(condition))
            user_lookup = tf.nn.embedding_lookup(user_embeddings, condition)
            item_factors = tf.layers.dense(user_lookup, units=num_factors, activation=None, kernel_initializer=glorot_uniform, name='item_factors')

            fake_data = tf.matmul(tf.squeeze(user_lookup, axis=1), item_embeddings, transpose_b=True)
            return fake_data

        self.autoencoder, self.generator = autoencoder, generator

    def fit(self, num_factors=10, emb_dim=32, epochs=300, batch_size=32, d_lr=1e-4, g_lr=1e-4, d_steps=1, g_steps=1,
            d_reg=0, g_reg=0, m=1, recon_coefficient=1e-2, allow_worse=None, freq=None, metrics=['MAP','NDCG','RMSE'], sample_every=None,
            validation_evaluator=None, validation_set=None):

        # Construct the model config
        self.config = dict(locals())
        del self.config['self']

        # First clear the session to save GPU memory
        tf.reset_default_graph()
        # Set fixed seed for the TF graph
        tf.set_random_seed(self.seed)

        self.build(num_factors, emb_dim)

        # Create optimizers
        opt_gen = tf.train.AdamOptimizer(learning_rate=g_lr)
        opt_disc = tf.train.AdamOptimizer(learning_rate=d_lr)

        # placeholders
        real_profile = tf.placeholder(dtype=tf.float32, shape=[None, self.num_items])
        self.user_id = tf.placeholder(dtype=tf.int32, shape=[None, 1])

        # generator ops
        self.fake_profile = self.generator(self.user_id)

        # autoencoder ops
        real_encoding, real_recon_loss = self.autoencoder(real_profile)
        fake_encoding, fake_recon_loss = self.autoencoder(self.fake_profile)

        # model parameters
        self.dvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='autoencoder')
        self.gvars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')

        with tf.variable_scope('dvars_best', reuse=tf.AUTO_REUSE):
            self.dvars_best = []
            for idx, var in enumerate(self.dvars):
                self.dvars_best.append(tf.get_variable('dva_r' + str(idx), shape=var.get_shape(), trainable=False))
        with tf.variable_scope('gvars_best', reuse=tf.AUTO_REUSE):
            self.gvars_best = []
            for idx, var in enumerate(self.gvars):
                self.gvars_best.append(tf.get_variable('gvar_' + str(idx), shape=var.get_shape(), trainable=False))

        # losses
        dloss = real_recon_loss + tf.maximum(0.0, m * real_recon_loss - fake_recon_loss) + \
                d_reg * tf.add_n([tf.nn.l2_loss(var) for var in self.dvars])
        gloss = (1 - recon_coefficient) * fake_recon_loss + \
                recon_coefficient * tf.losses.mean_squared_error(real_encoding, fake_encoding) + \
                g_reg * tf.add_n([tf.nn.l2_loss(var) for var in self.gvars])

        # update ops
        dtrain = opt_disc.minimize(dloss, var_list=self.dvars)
        gtrain = opt_gen.minimize(gloss, var_list=self.gvars)

        self.sess = tf.Session()
        self.sess.run(tf.initialize_all_variables())

        self._stop_training = False
        if validation_evaluator is not None:
            early_stop = EarlyStoppingScheduler(self, evaluator=validation_evaluator, allow_worse=allow_worse,
                                                freq=freq, metrics=metrics)

        all_users = np.array(range(self.num_users))
        step = batch_size

        train_g_loss = []
        train_d_loss = []

        if self.verbose:
            print('Starting training...')

        t_start = time.time()
        e_start = time.time()

        epoch = 1

        pbar = tqdm.tqdm(total=epochs, initial=1)

        while not self._stop_training and epoch < epochs + 1:
            batch_d_loss = []
            batch_g_loss = []
            np.random.shuffle(all_users)
            for _ in range(d_steps):
                start_idx = 0
                while start_idx < len(all_users):
                    end_idx = start_idx + step
                    if end_idx > len(all_users):
                        end_idx = len(all_users)

                    uids = all_users[start_idx: end_idx]
                    real_histories = self.URM_train[uids].toarray()

                    _, _dloss = self.sess.run([dtrain, dloss], {real_profile: real_histories, self.user_id: uids.reshape(-1, 1)})
                    batch_d_loss.append(_dloss)
                    start_idx = end_idx

            for _ in range(g_steps):
                start_idx = 0
                while start_idx < len(all_users):
                    end_idx = start_idx + step
                    if end_idx > len(all_users):
                        end_idx = len(all_users)

                    uids = all_users[start_idx: end_idx]
                    real_histories = self.URM_train[uids].toarray()
                    _, _gloss = self.sess.run([gtrain, gloss], {real_profile: real_histories, self.user_id: uids.reshape(-1, 1)})
                    batch_g_loss.append(_gloss)
                    start_idx = end_idx

            mean_epoch_g_loss = np.mean(batch_g_loss)
            mean_epoch_d_loss = np.mean(batch_d_loss)

            train_g_loss.append(mean_epoch_g_loss)
            train_d_loss.append(mean_epoch_d_loss)

            if validation_set is not None and sample_every is not None and epoch % sample_every == 0:
                t_end = time.time()
                total = t_end-e_start
                print('Epoch : {:d}. Total: {:.2f} secs, {:.2f} secs/epoch.'.format(epoch, total, total/sample_every))
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()
                _, results_run_string = validation_evaluator.evaluateRecommender(self)
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()
                print(results_run_string)
                e_start = time.time()

            if validation_evaluator is not None:
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()
                early_stop(epoch)
                if self.mode == 'item':
                    self.URM_train = self.URM_train.T.tocsr()

                if self._stop_training:
                    print('Training stopped, epoch:', epoch)

            epoch += 1
            pbar.update()
        pbar.close()

        t_end = time.time()
        if self.verbose:
            print('Training took {:.2f} seconds'.format(t_end - t_start))

        if self.mode == 'item':
            self.URM_train = self.URM_train.T.tocsr()

        return epoch

    def stop_fit(self):
        self._stop_training = True

    def save_current_model(self):
        for idx, var in enumerate(self.dvars):
            self.sess.run(self.dvars_best[idx].assign(var))
        for idx, var in enumerate(self.gvars):
            self.sess.run(self.gvars_best[idx].assign(var))

    def load_model(self):
        for idx, var in enumerate(self.dvars_best):
            self.sess.run(self.dvars[idx].assign(var))
        for idx, var in enumerate(self.gvars_best):
            self.sess.run(self.gvars[idx].assign(var))

    def _compute_item_score(self, user_id_array, items_to_compute=None):
        if self.mode == 'item':
            predictions = self.sess.run(self.fake_profile, {self.user_id: np.array(range(self.num_users)).reshape(-1, 1)})
            return predictions.transpose()[user_id_array]
        else:
            return self.sess.run(self.fake_profile, {self.user_id: user_id_array.reshape(-1, 1)})

In [ ]:
gan = GANAE(URM_train, mode='user')

In [ ]:
import numpy as np
gan = GANAE(URM_train, mode='user')
evaluatorValidation = EvaluatorHoldout(URM_validation,[5,10,100],exclude_seen=np.bool_(True))

In [ ]:
np.bool_(True)

True

In [ ]:
gan.fit(num_factors=250,
        emb_dim=461,
        d_reg=0.0001,
        g_reg=0,
        epochs=300,
        batch_size=512,
        g_lr=0.0001,
        d_lr=0.0001,
        d_steps=1,
        g_steps=1,
        recon_coefficient=0.1728565040890659,
        m=10,
        allow_worse=5,
        freq=5,
        validation_evaluator=evaluatorValidation,
        sample_every=10,
        validation_set=URM_validation)


  3%|▎         | 10/300 [00:26<10:02,  2.08s/it]

Epoch : 10. Total: 26.57 secs, 2.66 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0069868, RECALL: 0.0020052, MAP: 0.0035573, NDCG: 0.0029978, F1: 0.0031161, HIT_RATE: 0.0349338, RMSE: 0.9991330, 
CUTOFF: 10 - PRECISION: 0.0067881, RECALL: 0.0038425, MAP: 0.0024785, NDCG: 0.0043791, F1: 0.0049072, HIT_RATE: 0.0678808, RMSE: 0.9991330, 
CUTOFF: 100 - PRECISION: 0.0062368, RECALL: 0.0339409, MAP: 0.0022383, NDCG: 0.0177605, F1: 0.0105373, HIT_RATE: 0.6236755, RMSE: 0.9991330, 




  7%|▋         | 20/300 [01:37<11:43,  2.51s/it]

Epoch : 20. Total: 50.24 secs, 5.02 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0341722, RECALL: 0.0083679, MAP: 0.0184164, NDCG: 0.0133341, F1: 0.0134438, HIT_RATE: 0.1708609, RMSE: 0.9865417, 
CUTOFF: 10 - PRECISION: 0.0304636, RECALL: 0.0152595, MAP: 0.0126816, NDCG: 0.0189002, F1: 0.0203337, HIT_RATE: 0.3046358, RMSE: 0.9865417, 
CUTOFF: 100 - PRECISION: 0.0196689, RECALL: 0.1034032, MAP: 0.0103589, NDCG: 0.0597673, F1: 0.0330509, HIT_RATE: 1.9668874, RMSE: 0.9865417, 




 10%|█         | 30/300 [02:51<11:53,  2.64s/it]

...

Epoch : 290. Total: 44.40 secs, 4.44 secs/epoch.
CUTOFF: 5 - PRECISION: 0.2052649, RECALL: 0.0981321, MAP: 0.1397368, NDCG: 0.1234053, F1: 0.1327836, HIT_RATE: 1.0263245, RMSE: 0.7934643, 
CUTOFF: 10 - PRECISION: 0.1811589, RECALL: 0.1645227, MAP: 0.1189206, NDCG: 0.1679302, F1: 0.1724405, HIT_RATE: 1.8115894, RMSE: 0.7934643, 
CUTOFF: 100 - PRECISION: 0.0784702, RECALL: 0.5454796, MAP: 0.1344535, NDCG: 0.3424797, F1: 0.1372030, HIT_RATE: 7.8470199, RMSE: 0.7934643, 




100%|██████████| 300/300 [31:57<00:00,  2.31s/it]

Epoch : 300. Total: 44.13 secs, 4.41 secs/epoch.
CUTOFF: 5 - PRECISION: 0.2047351, RECALL: 0.0980070, MAP: 0.1388544, NDCG: 0.1229147, F1: 0.1325582, HIT_RATE: 1.0236755, RMSE: 0.7956042, 
CUTOFF: 10 - PRECISION: 0.1800166, RECALL: 0.1627580, MAP: 0.1178788, NDCG: 0.1666120, F1: 0.1709528, HIT_RATE: 1.8001656, RMSE: 0.7956042, 
CUTOFF: 100 - PRECISION: 0.0780712, RECALL: 0.5407580, MAP: 0.1332479, NDCG: 0.3399497, F1: 0.1364435, HIT_RATE: 7.8071192, RMSE: 0.7956042, 




301it [32:39,  6.53s/it]


301

In [ ]:
evaluator = EvaluatorHoldout(URM_test, [5,10,100], exclude_seen=True)

<ipython-input-11-51a3e3fe9a07>:90: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)


In [ ]:
results_dic, results_run_string = evaluator.evaluateRecommender(gan)
print(results_run_string)

CUTOFF: 5 - PRECISION: 0.2291722, RECALL: 0.0955034, MAP: 0.1501598, NDCG: 0.1286003, F1: 0.1348221, HIT_RATE: 1.1458609, RMSE: 0.7964392, 
CUTOFF: 10 - PRECISION: 0.2022185, RECALL: 0.1607081, MAP: 0.1250872, NDCG: 0.1757140, F1: 0.1790894, HIT_RATE: 2.0221854, RMSE: 0.7964392, 
CUTOFF: 100 - PRECISION: 0.0858295, RECALL: 0.5395107, MAP: 0.1410441, NDCG: 0.3594203, F1: 0.1480983, HIT_RATE: 8.5829470, RMSE: 0.7964392, 



In [ ]:
recommended_items_batch_list, scores_batch = gan.recommend(user_id_array=np.array([i for i in range(1,101)]),
                                                                      remove_seen_flag=True,
                                                                      cutoff = 10,
                                                                      remove_top_pop_flag=False,
                                                                      remove_CustomItems_flag=False,
                                                                      return_scores = True
                                                                     )

In [ ]:
np.array(recommended_items_batch_list)

array([[ 699,  367, 1449,  460,  737, 1476,  309, 1877, 1781,  253],
       [1108,    0, 2599, 2710, 1186, 2426, 2586, 2775, 2958,  578],
       [1106, 1212,  106, 1199, 1148,  443,  802, 1110, 1478, 2374],
       [2557, 2202,  287,  216, 2143, 1536, 1022,   49,  443, 2203],
       [ 858, 1767,  838, 3277,  573,  347,   38, 1173, 3321,  525],
       [ 575,  699,  367, 1449, 1877,  802,   20, 3341, 1120, 1732],
       [1294, 3186, 2099,   60, 2374,  579,  309, 2748, 1260,  291],
       [1563, 1848,  287, 3341, 2203, 2785,  346, 3510,  593, 2198],
       [1178,  963, 2426, 1900, 3256, 2213, 3165,  357,  574, 1767],
       [1398, 2131, 2400, 2651,   38, 1504,  216, 3029, 3039,    0],
       [ 851, 1843, 1104, 1839, 1123, 1135,  847, 1138, 1117,  593],
       [ 699,  339, 1275, 1848, 1428,    9, 3256, 1277, 2012,  202],
       [2897, 2866, 1123, 1118, 2501, 2511, 2505, 2748, 3321, 2958],
       [1492, 2651, 2748, 1445,  339,  367, 2683,  593, 1618, 1428],
       [2384, 1731, 2490, 2501, 13

In [ ]:
import pandas as pd
df=pd.DataFrame(np.array(recommended_items_batch_list))
df.to_csv("recommend_list.csv")

In [ ]:
print(np.array(URM_train[1,466]))

1.0


Training uisng Random walk Graph sampling

In [ ]:
from tqdm import tqdm

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

ratings = pd.read_csv("/content/data/ml-1m/ratings.dat", sep="::", header=None,
                      names=["user_id", "movie_id", "rating", "timestamp"])

# Build a user-item graph
G = nx.Graph()
G.add_nodes_from(ratings["user_id"].unique(), bipartite=0)
G.add_nodes_from(ratings["movie_id"].unique(), bipartite=1)
G.add_weighted_edges_from(ratings[["user_id", "movie_id", "rating"]].values)

# Define the random walk length and number of sequences per user
walk_length = 100
num_walks = 20


<ipython-input-1-3e2fcd2545d4>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("/content/data/ml-1m/ratings.dat", sep="::", header=None,


In [ ]:

# Perform random walks for each user
sequences = []
for user in tqdm(ratings["user_id"].unique()):
    for i in range(num_walks):
        sequence = [user]
        for j in range(walk_length):
            neighbors = list(G.neighbors(sequence[-1]))
            if len(neighbors) > 0:
                next_node = np.random.choice(neighbors)
                sequence.append(next_node)
            else:
                break
        sequences.append(sequence)
np.random.shuffle(sequences)
train_ratio = 0.7
train_size = int(len(sequences) * train_ratio)
train_sequences = sequences[:train_size]
validationsequences=sequences[train_size:train_size+int(len(sequences)*0.1)]
test_sequences = sequences[train_size+int(len(sequences)*0.1):]

100%|██████████| 6040/6040 [43:20<00:00,  2.32it/s]


In [ ]:
pd.DataFrame(sequences).to_csv("randomwalk_sequences.csv")

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

train_array=np.array(train_sequences)
user_idx = np.unique(train_array[:, 0], return_inverse=True)[1]
item_idx = np.unique(train_array[:, 1], return_inverse=True)[1]

user_item_matrix_train = csr_matrix((train_array[:, 2], (user_idx, item_idx)))

validation_array=np.array(validationsequences)
user_idx = np.unique(validation_array[:, 0], return_inverse=True)[1]
item_idx = np.unique(validation_array[:, 1], return_inverse=True)[1]

user_item_matrix_validation = csr_matrix((validation_array[:, 2], (user_idx, item_idx)))


test_array=np.array(test_sequences)
user_idx = np.unique(test_array[:, 0], return_inverse=True)[1]
item_idx = np.unique(test_array[:, 1], return_inverse=True)[1]

user_item_matrix_test = csr_matrix((test_array[:, 2], (user_idx, item_idx)))

In [ ]:
URM_train_matrix = np.zeros((6040, 3706))
URM_train_matrix[:, :] = user_item_matrix_train[:, 0:3706].toarray()
URM_train_matrix=csr_matrix(URM_train_matrix)

In [ ]:
URM_validation_matrix = np.zeros((5284, 3427))

URM_validation_matrix[:, :] = user_item_matrix_validation[:, 0:3427].toarray()
URM_validation_matrix=csr_matrix(URM_validation_matrix)

In [ ]:
URM_test_matrix = np.zeros((5966, 3427))

URM_test_matrix[:, :] = user_item_matrix_test[:, 0:3427].toarray()
URM_validation_matrix=csr_matrix(URM_validation_matrix)

In [ ]:
user_item_matrix_train.shape

(6040, 5486)

In [ ]:
evaluatorValidation = EvaluatorHoldout(URM_validation,[5,10,20,50,80,100],exclude_seen=np.bool_(True))

<ipython-input-27-6ceb33196298>:90: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)


In [ ]:
gan1 = GANAE(URM_train_matrix, mode='user')
gan1.fit(num_factors=250,
        emb_dim=461,
        d_reg=0.0001,
        g_reg=0,
        epochs=250,
        batch_size=512,
        g_lr=0.0001,
        d_lr=0.0001,
        d_steps=1,
        g_steps=1,
        recon_coefficient=0.1728565040890659,
        m=10,
        allow_worse=5,
        freq=5,
        validation_evaluator=evaluatorValidation,
        sample_every=10,
        validation_set=URM_validation_matrix)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
  4%|▍         | 10/250 [02:02<45:55, 11.48s/it]

Epoch : 10. Total: 130.56 secs, 13.06 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0041060, RECALL: 0.0011320, MAP: 0.0018707, NDCG: 0.0015943, F1: 0.0017747, HIT_RATE: 0.0205298, RMSE: 1.0000228, 
CUTOFF: 10 - PRECISION: 0.0043046, RECALL: 0.0026558, MAP: 0.0014298, NDCG: 0.0026786, F1: 0.0032849, HIT_RATE: 0.0430464, RMSE: 1.0000228, 
CUTOFF: 20 - PRECISION: 0.0043212, RECALL: 0.0053283, MAP: 0.0011952, NDCG: 0.0042028, F1: 0.0047722, HIT_RATE: 0.0864238, RMSE: 1.0000228, 
CUTOFF: 50 - PRECISION: 0.0044139, RECALL: 0.0141977, MAP: 0.0012603, NDCG: 0.0080848, F1: 0.0067342, HIT_RATE: 0.2206954, RMSE: 1.0000228, 
CUTOFF: 80 - PRECISION: 0.0044329, RECALL: 0.0219202, MAP: 0.0013835, NDCG: 0.0111518, F1: 0.0073745, HIT_RATE: 0.3546358, RMSE: 1.0000228, 
CUTOFF: 100 - PRECISION: 0.0044321, RECALL: 0.0274748, MAP: 0.0014625, NDCG: 0.0131106, F1: 0.0076329, HIT_RATE: 0.4432119, RMSE: 1.0000228, 



  8%|▊         | 20/250 [05:51<49:42, 12.97s/it]

Epoch : 20. Total: 178.33 secs, 17.83 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0042715, RECALL: 0.0014223, MAP: 0.0019854, NDCG: 0.0018281, F1: 0.0021340, HIT_RATE: 0.0213576, RMSE: 0.9985527, 
CUTOFF: 10 - PRECISION: 0.0044040, RECALL: 0.0029413, MAP: 0.0015612, NDCG: 0.0029126, F1: 0.0035270, HIT_RATE: 0.0440397, RMSE: 0.9985527, 
CUTOFF: 20 - PRECISION: 0.0046772, RECALL: 0.0061490, MAP: 0.0013790, NDCG: 0.0047213, F1: 0.0053130, HIT_RATE: 0.0935430, RMSE: 0.9985527, 
CUTOFF: 50 - PRECISION: 0.0046722, RECALL: 0.0146493, MAP: 0.0014218, NDCG: 0.0086018, F1: 0.0070848, HIT_RATE: 0.2336093, RMSE: 0.9985527, 
CUTOFF: 80 - PRECISION: 0.0045757, RECALL: 0.0229893, MAP: 0.0015542, NDCG: 0.0117664, F1: 0.0076324, HIT_RATE: 0.3660596, RMSE: 0.9985527, 
CUTOFF: 100 - PRECISION: 0.0046275, RECALL: 0.0292290, MAP: 0.0016416, NDCG: 0.0139491, F1: 0.0079900, HIT_RATE: 0.4627483, RMSE: 0.9985527, 



 12%|█▏        | 30/250 [09:37<46:38, 12.72s/it]

...

 24%|██▍       | 60/250 [21:22<41:29, 13.10s/it]

Epoch : 60. Total: 192.63 secs, 19.26 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0047020, RECALL: 0.0017168, MAP: 0.0021827, NDCG: 0.0021201, F1: 0.0025152, HIT_RATE: 0.0235099, RMSE: 0.8649181, 
CUTOFF: 10 - PRECISION: 0.0052483, RECALL: 0.0034806, MAP: 0.0017659, NDCG: 0.0034368, F1: 0.0041854, HIT_RATE: 0.0524834, RMSE: 0.8649181, 
CUTOFF: 20 - PRECISION: 0.0057864, RECALL: 0.0077385, MAP: 0.0016562, NDCG: 0.0058531, F1: 0.0066216, HIT_RATE: 0.1157285, RMSE: 0.8649181, 
CUTOFF: 50 - PRECISION: 0.0062881, RECALL: 0.0215063, MAP: 0.0018920, NDCG: 0.0118967, F1: 0.0097310, HIT_RATE: 0.3144040, RMSE: 0.8649181, 
CUTOFF: 80 - PRECISION: 0.0064901, RECALL: 0.0349789, MAP: 0.0021384, NDCG: 0.0170019, F1: 0.0109487, HIT_RATE: 0.5192053, RMSE: 0.8649181, 
CUTOFF: 100 - PRECISION: 0.0065248, RECALL: 0.0441346, MAP: 0.0022778, NDCG: 0.0201475, F1: 0.0113689, HIT_RATE: 0.6524834, RMSE: 0.8649181, 



 24%|██▍       | 61/250 [23:09<1:12:56, 23.16s/it]

Training stopped, epoch: 60


61

In [ ]:
evaluator = EvaluatorHoldout(URM_test, [5,10,20,50,80,100,300,500,1000], exclude_seen=True)

<ipython-input-27-6ceb33196298>:90: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  usersToEvaluate_mask = np.zeros(self.n_users, dtype=np.bool)


In [ ]:
results_dic, results_run_string = evaluator.evaluateRecommender(gan1)
print(results_run_string)

CUTOFF: 5 - PRECISION: 0.0050331, RECALL: 0.0015026, MAP: 0.0022886, NDCG: 0.0021175, F1: 0.0023143, HIT_RATE: 0.0251656, RMSE: 0.9839834, 
CUTOFF: 10 - PRECISION: 0.0053808, RECALL: 0.0030409, MAP: 0.0017190, NDCG: 0.0033290, F1: 0.0038858, HIT_RATE: 0.0538079, RMSE: 0.9839834, 
CUTOFF: 20 - PRECISION: 0.0055464, RECALL: 0.0063677, MAP: 0.0014580, NDCG: 0.0053339, F1: 0.0059287, HIT_RATE: 0.1109272, RMSE: 0.9839834, 
CUTOFF: 50 - PRECISION: 0.0058709, RECALL: 0.0173254, MAP: 0.0015661, NDCG: 0.0105077, F1: 0.0087699, HIT_RATE: 0.2935430, RMSE: 0.9839834, 
CUTOFF: 80 - PRECISION: 0.0061341, RECALL: 0.0288262, MAP: 0.0017703, NDCG: 0.0151514, F1: 0.0101156, HIT_RATE: 0.4907285, RMSE: 0.9839834, 
CUTOFF: 100 - PRECISION: 0.0060861, RECALL: 0.0353743, MAP: 0.0018756, NDCG: 0.0176454, F1: 0.0103854, HIT_RATE: 0.6086093, RMSE: 0.9839834, 
CUTOFF: 300 - PRECISION: 0.0058813, RECALL: 0.1014439, MAP: 0.0026020, NDCG: 0.0389709, F1: 0.0111181, HIT_RATE: 1.7644040, RMSE: 0.9839834, 
CUTOFF: 500 

In [ ]:
!unzip "/content/data2.zip"

Archive:  /content/data2.zip
  inflating: data2.csv               


Training Using Distill -cf generated synthesized data

In [ ]:
import pandas as pd
df=pd.read_csv("/content/data2.csv")
df = df.drop(df.columns[0], axis=1)
df=df.round()
df.head()


,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
URM_train=csr_matrix(df)
gan2 = GANAE(URM_train, mode='user')
gan2.fit(num_factors=250,
        emb_dim=461,
        d_reg=0.0001,
        g_reg=0,
        epochs=250,
        batch_size=32,
        g_lr=0.0001,
        d_lr=0.0001,
        d_steps=1,
        g_steps=1,
        recon_coefficient=0.1728565040890659,
        m=10,
        allow_worse=5,
        freq=5,
        validation_evaluator=evaluatorValidation,
        sample_every=10,
        validation_set=URM_validation)


  4%|▍         | 10/250 [00:23<07:36,  1.90s/it]

Epoch : 10. Total: 24.10 secs, 2.41 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0108278, RECALL: 0.0038336, MAP: 0.0043967, NDCG: 0.0045093, F1: 0.0056624, HIT_RATE: 0.0541391, RMSE: nan, 
CUTOFF: 10 - PRECISION: 0.0092384, RECALL: 0.0061430, MAP: 0.0031709, NDCG: 0.0062004, F1: 0.0073792, HIT_RATE: 0.0923841, RMSE: nan, 
CUTOFF: 100 - PRECISION: 0.0091209, RECALL: 0.0618678, MAP: 0.0036053, NDCG: 0.0289534, F1: 0.0158980, HIT_RATE: 0.9120861, RMSE: nan, 




  8%|▊         | 20/250 [01:28<08:53,  2.32s/it]

Epoch : 20. Total: 43.81 secs, 4.38 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0306291, RECALL: 0.0141731, MAP: 0.0193073, NDCG: 0.0201423, F1: 0.0193790, HIT_RATE: 0.1531457, RMSE: nan, 
CUTOFF: 10 - PRECISION: 0.0279801, RECALL: 0.0249493, MAP: 0.0150467, NDCG: 0.0273472, F1: 0.0263780, HIT_RATE: 0.2798013, RMSE: nan, 
CUTOFF: 100 - PRECISION: 0.0206242, RECALL: 0.1601681, MAP: 0.0169240, NDCG: 0.0817239, F1: 0.0365429, HIT_RATE: 2.0624172, RMSE: nan, 




 12%|█▏        | 30/250 [02:32<08:23,  2.29s/it]

...

Epoch : 240. Total: 43.24 secs, 4.32 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0214238, RECALL: 0.0093768, MAP: 0.0129443, NDCG: 0.0131670, F1: 0.0130443, HIT_RATE: 0.1071192, RMSE: nan, 
CUTOFF: 10 - PRECISION: 0.0222517, RECALL: 0.0196974, MAP: 0.0101466, NDCG: 0.0198491, F1: 0.0208968, HIT_RATE: 0.2225166, RMSE: nan, 
CUTOFF: 100 - PRECISION: 0.0156308, RECALL: 0.1155128, MAP: 0.0110149, NDCG: 0.0590140, F1: 0.0275356, HIT_RATE: 1.5630795, RMSE: nan, 




100%|██████████| 250/250 [25:50<00:00,  2.27s/it]

Epoch : 250. Total: 43.24 secs, 4.32 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0215563, RECALL: 0.0093609, MAP: 0.0129529, NDCG: 0.0131236, F1: 0.0130533, HIT_RATE: 0.1077815, RMSE: nan, 
CUTOFF: 10 - PRECISION: 0.0223179, RECALL: 0.0196920, MAP: 0.0101791, NDCG: 0.0198851, F1: 0.0209229, HIT_RATE: 0.2231788, RMSE: nan, 
CUTOFF: 100 - PRECISION: 0.0156457, RECALL: 0.1155409, MAP: 0.0110296, NDCG: 0.0591015, F1: 0.0275595, HIT_RATE: 1.5645695, RMSE: nan, 




251it [26:30,  6.36s/it]


251

In [ ]:
import pandas as pd
df=pd.read_csv("/content/data1.csv")


In [ ]:
df = df.drop(df.columns[0], axis=1)

In [ ]:
df=df.round()

In [ ]:
df.iloc[0].sum()

169.0

In [ ]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,3696,3697,3698,3699,3700,3701,3702,3703,3704,3705
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
URM_train=csr_matrix(df)

In [ ]:
URM_train.shape

(500, 3706)

In [ ]:
gan2 = GANAE(URM_train, mode='user')
gan2.fit(num_factors=250,
        emb_dim=461,
        d_reg=0.0001,
        g_reg=0,
        epochs=250,
        batch_size=32,
        g_lr=0.0001,
        d_lr=0.0001,
        d_steps=1,
        g_steps=1,
        recon_coefficient=0.1728565040890659,
        m=10,
        allow_worse=5,
        freq=5,
        validation_evaluator=evaluatorValidation,
        sample_every=10,
        validation_set=URM_validation)

  4%|▍         | 10/250 [00:33<11:39,  2.91s/it]

Epoch : 10. Total: 35.36 secs, 3.54 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0087417, RECALL: 0.0026442, MAP: 0.0039680, NDCG: 0.0035764, F1: 0.0040603, HIT_RATE: 0.0437086, RMSE: nan, 
CUTOFF: 20 - PRECISION: 0.0065977, RECALL: 0.0067990, MAP: 0.0020355, NDCG: 0.0064488, F1: 0.0066968, HIT_RATE: 0.1319536, RMSE: nan, 



  8%|▊         | 20/250 [01:42<13:34,  3.54s/it]

Epoch : 20. Total: 50.95 secs, 5.10 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0119868, RECALL: 0.0034885, MAP: 0.0060805, NDCG: 0.0052033, F1: 0.0054042, HIT_RATE: 0.0599338, RMSE: nan, 
CUTOFF: 20 - PRECISION: 0.0108444, RECALL: 0.0140421, MAP: 0.0036853, NDCG: 0.0119684, F1: 0.0122378, HIT_RATE: 0.2168874, RMSE: nan, 



 12%|█▏        | 30/250 [02:51<12:09,  3.31s/it]

...

Epoch : 240. Total: 48.88 secs, 4.89 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0366556, RECALL: 0.0158931, MAP: 0.0198652, NDCG: 0.0206680, F1: 0.0221727, HIT_RATE: 0.1832781, RMSE: nan, 
CUTOFF: 20 - PRECISION: 0.0346937, RECALL: 0.0603250, MAP: 0.0154013, NDCG: 0.0455479, F1: 0.0440523, HIT_RATE: 0.6938742, RMSE: nan, 



100%|██████████| 250/250 [26:49<00:00,  3.31s/it]

Epoch : 250. Total: 50.27 secs, 5.03 secs/epoch.
CUTOFF: 5 - PRECISION: 0.0365894, RECALL: 0.0159107, MAP: 0.0198479, NDCG: 0.0206629, F1: 0.0221776, HIT_RATE: 0.1829470, RMSE: nan, 
CUTOFF: 20 - PRECISION: 0.0346523, RECALL: 0.0602565, MAP: 0.0153772, NDCG: 0.0454814, F1: 0.0440007, HIT_RATE: 0.6930464, RMSE: nan, 



251it [27:24,  6.58s/it]


251

In [ ]:
num_users

6040

In [ ]:
results_dic, results_run_string = evaluator.evaluateRecommender(gan2)
print(results_run_string)

CUTOFF: 5 - PRECISION: 0.0423510, RECALL: 0.0160711, MAP: 0.0221049, NDCG: 0.0221876, F1: 0.0233003, HIT_RATE: 0.2117550, RMSE: nan, 
CUTOFF: 10 - PRECISION: 0.0387417, RECALL: 0.0292998, MAP: 0.0163899, NDCG: 0.0316101, F1: 0.0333656, HIT_RATE: 0.3874172, RMSE: nan, 
CUTOFF: 20 - PRECISION: 0.0391142, RECALL: 0.0605958, MAP: 0.0155210, NDCG: 0.0489125, F1: 0.0475410, HIT_RATE: 0.7822848, RMSE: nan, 
CUTOFF: 50 - PRECISION: 0.0313775, RECALL: 0.1162087, MAP: 0.0171043, NDCG: 0.0736320, F1: 0.0494130, HIT_RATE: 1.5688742, RMSE: nan, 
CUTOFF: 100 - PRECISION: 0.0259768, RECALL: 0.1818541, MAP: 0.0195648, NDCG: 0.0986566, F1: 0.0454599, HIT_RATE: 2.5976821, RMSE: nan, 



infinite auto encoder

In [ ]:
train_x=ret['train_matrix']
sample_user=jnp.array(ret['train_matrix'][ret['train_matrix'].getnnz(1)>0].todense())

In [ ]:
!pip install neural_tangents

In [ ]:
import jax
import functools
from jax import scipy as sp
from jax import numpy as jnp
from neural_tangents import stax

a,b,kernel_fn=stax.serial(stax.Flatten(),
    stax.Dense(1024,W_std=2*0.5,b_std=0.1,parameterization='ntk'),
    stax.Relu(),
    stax.Dense(num_items, W_std=2*0.5, b_std=0.1, parameterization='ntk'))
kernel_fn=functools.partial(kernel_fn,get='ntk')
@jax.jit
def kernelized_rr(X_train, X_predict, reg):
    K_train = kernel_fn(X_train, X_train)
    K_predict = kernel_fn(X_predict, X_train)
    K_reg = (K_train + jnp.abs(reg) * jnp.trace(K_train) * jnp.eye(K_train.shape[0]) / K_train.shape[0])     
    return jnp.dot(K_predict, sp.linalg.solve(K_reg, X_train, assume_a='pos'))

In [ ]:
preds=kernelized_rr(sample_user,(ret['train_matrix']).todense(),reg=5.0)

In [ ]:
logits=np.array(preds)
train_positive_list = list(map(list, ret['train_positive_set']))
for b in range(len(logits)):logits[b][train_positive_list[b]]=-float(1e6)
indices = (-logits).argsort()[:, :100].tolist()
hr=0
for i in range(100):
  for b in range(len(logits)):
    num_pos = float(len((ret['val_positive_set']+ret['test_positive_set'])[b]))

    hr += float(len(set(indices[b][:100]) &(ret['val_positive_set']+ret['test_positive_set'])[b])) / float(min(num_pos, 100))
hr/=num_users
print("hit rate for top @100:",hr)

hit rate for top @100: 58.43498829318017


In [ ]:
dcg,idcg=0.0,0.0
for i in range(len(logits)):
  for at, pred in enumerate(indices[b][:100]):
      if pred in (ret['test_positive_set'])[b]: 
          dcg += 1.0 / np.log2(at + 2)
      if at < num_pos: 
          idcg += 1.0 / np.log2(at + 2)

In [ ]:
print('ndcg @100:',dcg/idcg)

ndcg @100: 0.47835283232987585


In [ ]:
preds

Array([[ 6.1822701e-01,  7.5850978e-02, -4.4492036e-03, ...,
        -3.3333562e-03, -5.0569102e-03, -1.0355160e-02],
       [ 1.8577071e-01,  1.8658340e-02,  2.4700627e-02, ...,
        -3.3351779e-04,  4.2843632e-03,  2.5685519e-02],
       [ 1.6359642e-01,  5.3604096e-02,  2.9977232e-02, ...,
         2.9501319e-03,  1.1231005e-04, -1.8003672e-02],
       ...,
       [ 6.3653909e-02, -4.4144541e-03,  2.2865012e-03, ...,
         1.0322370e-03, -1.4050305e-03,  1.2156427e-02],
       [ 1.4754696e-01,  4.7182813e-02,  1.6691655e-02, ...,
         4.1483752e-03, -7.0306063e-03,  9.4670355e-03],
       [ 6.4318651e-01,  9.8272264e-03, -2.2913679e-02, ...,
         2.8302886e-02,  2.7466364e-02,  7.5228065e-02]], dtype=float32)